---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [7]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [18]:
def graph_identification(debug=False):
    
    # Your Code Here
    if debug:
        print(nx.info(P1_Graphs[0]), "\n")
        print(nx.info(P1_Graphs[1]), "\n")
        print(nx.info(P1_Graphs[2]), "\n")
        print(nx.info(P1_Graphs[3]), "\n")
        print(nx.info(P1_Graphs[4]), "\n")
        
        for cnt, G in enumerate(P1_Graphs):
            print("G[{}]: Avg. clu= {}\n      Avg. SPL= {}".format(
                cnt, nx.average_clustering(G), nx.average_shortest_path_length(G)))

    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']       # Your Answer Here

# graph_identification(True)

Name: barabasi_albert_graph(1000,2)
Type: Graph
Number of nodes: 1000
Number of edges: 1996
Average degree:   3.9920 

Name: watts_strogatz_graph(1000,10,0.05)
Type: Graph
Number of nodes: 1000
Number of edges: 5000
Average degree:  10.0000 

Name: watts_strogatz_graph(750,5,0.075)
Type: Graph
Number of nodes: 750
Number of edges: 1500
Average degree:   4.0000 

Name: barabasi_albert_graph(750,4)
Type: Graph
Number of nodes: 750
Number of edges: 2984
Average degree:   7.9573 

Name: watts_strogatz_graph(750,4,1)
Type: Graph
Number of nodes: 750
Number of edges: 1500
Average degree:   4.0000 

G[0]: Avg. clu= 0.03167539146454044
      Avg. SPL= 4.099161161161161
G[1]: Avg. clu= 0.5642419635919628
      Avg. SPL= 5.089871871871872
G[2]: Avg. clu= 0.4018222222222227
      Avg. SPL= 9.378702269692925
G[3]: Avg. clu= 0.03780379975223251
      Avg. SPL= 3.1048046283934134
G[4]: Avg. clu= 0.0033037037037037037
      Avg. SPL= 5.0785509568313305


---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [21]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [26]:
def salary_predictions(debug=False):
    
    # Your Code Here
    from sklearn import preprocessing
    from sklearn import metrics
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    
    # generate dataset
    df = pd.DataFrame(index=G.nodes())

    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    
    # Assign different measurements into dataframe
    df['degree'] = pd.Series(nx.degree_centrality(G))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G))
    df['betweenness'] = pd.Series(nx.betweenness_centrality(G))
    
    # Split the dataframe into two sub-dataframes: df_train & df_pred
    df_train = df.dropna()
    df_pred = df[np.isnan(df['ManagementSalary'])]
    
    selected_attrs = ['Department', 'degree', 'closeness', 'betweenness', 'clustering']
#     selected_attrs = ['degree', 'closeness', 'betweenness', 'clustering']
    
    X = df_train[selected_attrs]
    y = df_train['ManagementSalary']
    
    # split into training and test data w/ given training dataframe
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Scale transformation
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Traing data with Logistic Regression Model
    clf = LogisticRegression()

    clf.fit(X_train, y_train)
    
    # check with training dataset with various metrics
    if debug:
        # predict the X_test
        pred1 = clf.predict(X_test)
        pred2 = clf.predict_proba(X_test)
        
        print("\nAccuracy: {}".format(metrics.accuracy_score(y_test, pred1)))
        print("\nROC_AUC: {}".format(metrics.roc_auc_score(y_test, pred2[:, 1])))
        print("\nConfusiion matrix: \n{}".format(metrics.confusion_matrix(y_test, pred1)))
        print("\nCalssification report: \n{}".format(metrics.classification_report(y_test, pred1)))
        
    # Generte result for autograder
    pred = clf.predict_proba(df_pred[selected_attrs])
    
    rlt = pd.Series(data=pred[:, 1], index=df_pred.index)
    
    return rlt # Your Answer Here

# salary_predictions(True)


Accuracy: 0.9337748344370861

ROC_AUC: 0.9351648351648352

Confusiion matrix: 
[[130   0]
 [ 10  11]]

Calssification report: 
             precision    recall  f1-score   support

        0.0       0.93      1.00      0.96       130
        1.0       1.00      0.52      0.69        21

avg / total       0.94      0.93      0.92       151



1       0.181858
2       0.208064
5       0.182571
8       0.225330
14      0.178320
18      0.200068
27      0.212109
30      0.227358
31      0.217607
34      0.266029
37      0.208985
40      0.202293
45      0.207783
54      0.250662
55      0.259667
60      0.208508
62      0.199358
65      0.178310
77      0.216167
79      0.201420
97      0.241895
101     0.183273
103     0.222051
108     0.195658
113     0.206024
122     0.177869
141     0.191647
142     0.207687
144     0.244066
145     0.233533
          ...   
913     0.192171
914     0.234379
915     0.089904
918     0.270603
923     0.234159
926     0.272085
931     0.221647
934     0.163733
939     0.157162
944     0.092295
945     0.242660
947     0.364782
950     0.172634
951     0.222070
953     0.171792
959     0.096826
962     0.091681
963     0.213835
968     0.349565
969     0.350890
974     0.305949
984     0.161642
987     0.305338
989     0.351414
991     0.354294
992     0.101191
994     0.097977
996     0.0929

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [ ]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [ ]:
def new_connections_predictions():
    
    # Your Code Here
    
    return # Your Answer Here